In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

## Feature Selection

Important features for our objective:

### Essential Features:
1. **Transaction_Amount** - Fraud often involves unusual amounts
2. **Account_Balance** - Ratio of transaction to balance might be important
3. **Device_Type** - Certain devices may be more risky
4. **Location** - Unusual locations may indicate fraud
5. **Merchant_Category** - Some categories are higher risk
6. **IP_Address_Flag** - Direct indicator of suspicious activity
7. **Daily_Transaction_Count** - Unusually high activity may indicate fraud
8. **Avg_Transaction_Amount_7d** - Deviation from normal behavior
9. **Failed_Transaction_Count_7d** - Multiple failures may indicate fraud attempts
10. **Transaction_Distance** - Distance from user's usual location
11. **Authentication_Method** - Some methods are more secure
12. **Risk_Score** - Pre-calculated risk metric
13. **Is_Weekend** - Fraud patterns may differ on weekends

### Features to potentially drop:
- **Transaction_ID** - Just an identifier
- **User_ID** - May not be useful unless doing user-specific analysis
- **Timestamp** - Could extract features from it (hour, day of week)
- **Card_Type** - May not be strongly correlated with fraud
- **Card_Age** - Less likely to be predictive

### Extract time-based features

In [25]:
df = pd.read_csv('../assets/synthetic_fraud_dataset.csv')

In [26]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['Is_Night'] = ((df['Hour'] >= 22) | (df['Hour'] <= 6)).astype(int)

In [27]:
df

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label,Hour,DayOfWeek,Is_Night
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,...,Amex,65,883.17,Biometric,0.8494,0,0,19,0,0
1,TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-07 04:01:00,75725.25,Mobile,New York,Clothing,0,...,Mastercard,186,2203.36,Password,0.0959,0,1,4,2,1
2,TXN_199,USER_2734,28.96,Online,2023-06-20 15:25:00,1588.96,Tablet,Mumbai,Restaurants,0,...,Visa,226,1909.29,Biometric,0.8400,0,1,15,1,0
3,TXN_12447,USER_2617,254.32,ATM Withdrawal,2023-12-07 00:31:00,76807.20,Tablet,New York,Clothing,0,...,Visa,76,1311.86,OTP,0.7935,0,1,0,3,1
4,TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44:00,92354.66,Mobile,Mumbai,Electronics,0,...,Mastercard,140,966.98,Password,0.3819,1,1,23,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,TXN_11284,USER_4796,45.05,Online,2023-01-29 18:38:00,76960.11,Mobile,Tokyo,Clothing,0,...,Amex,98,1537.54,PIN,0.1493,1,0,18,6,0
49996,TXN_44732,USER_1171,126.15,POS,2023-05-09 08:55:00,28791.75,Mobile,Tokyo,Clothing,0,...,Visa,93,2555.72,Biometric,0.3653,0,1,8,1,0
49997,TXN_38158,USER_2510,72.02,Online,2023-01-30 19:32:00,29916.41,Laptop,Mumbai,Clothing,0,...,Visa,114,4686.59,Biometric,0.5195,0,0,19,0,0
49998,TXN_860,USER_2248,64.89,Bank Transfer,2023-03-09 19:47:00,67895.67,Mobile,Tokyo,Electronics,0,...,Discover,72,4886.92,Biometric,0.7063,0,1,19,3,0


### Create transaction amount to balance ratio

In [28]:
df['Amount_to_Balance_Ratio'] = df['Transaction_Amount'] / df['Account_Balance']

### Create deviation from average transaction amount

In [29]:
df['Amount_Deviation'] = df['Transaction_Amount'] - df['Avg_Transaction_Amount_7d']

### Create binary flag for high-risk categories

In [30]:
high_risk_categories = ['Travel', 'Electronics', 'Clothing']
df['High_Risk_Category'] = df['Merchant_Category'].isin(high_risk_categories).astype(int)

### Drop original timestamp and IDs

In [31]:
df = df.drop(['Transaction_ID', 'User_ID', 'Timestamp', 'Card_Type', 'Card_Age'], axis=1)

In [32]:
df

,Transaction_Amount,Transaction_Type,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,Previous_Fraudulent_Activity,Daily_Transaction_Count,Avg_Transaction_Amount_7d,...,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label,Hour,DayOfWeek,Is_Night,Amount_to_Balance_Ratio,Amount_Deviation,High_Risk_Category
0,39.79,POS,93213.17,Laptop,Sydney,Travel,0,0,7,437.63,...,Biometric,0.8494,0,0,19,0,0,0.000427,-397.84,1
1,1.19,Bank Transfer,75725.25,Mobile,New York,Clothing,0,0,13,478.76,...,Password,0.0959,0,1,4,2,1,0.000016,-477.57,1
2,28.96,Online,1588.96,Tablet,Mumbai,Restaurants,0,0,14,50.01,...,Biometric,0.8400,0,1,15,1,0,0.018226,-21.05,0
3,254.32,ATM Withdrawal,76807.20,Tablet,New York,Clothing,0,0,8,182.48,...,OTP,0.7935,0,1,0,3,1,0.003311,71.84,1
4,31.28,POS,92354.66,Mobile,Mumbai,Electronics,0,1,14,328.69,...,Password,0.3819,1,1,23,5,1,0.000339,-297.41,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,45.05,Online,76960.11,Mobile,Tokyo,Clothing,0,0,2,389.00,...,PIN,0.1493,1,0,18,6,0,0.000585,-343.95,1
49996,126.15,POS,28791.75,Mobile,Tokyo,Clothing,0,0,13,434.95,...,Biometric,0.3653,0,1,8,1,0,0.004381,-308.80,1
49997,72.02,Online,29916.41,Laptop,Mumbai,Clothing,0,1,1,369.15,...,Biometric,0.5195,0,0,19,0,0,0.002407,-297.13,1
49998,64.89,Bank Transfer,67895.67,Mobile,Tokyo,Electronics,0,0,13,242.29,...,Biometric,0.7063,0,1,19,3,0,0.000956,-177.40,1


### Data Preprocessing

In [33]:
X = df.drop('Fraud_Label', axis=1)
y = df['Fraud_Label']

#### Define categorical and numerical features

In [34]:
categorical_features = ['Transaction_Type', 'Device_Type', 'Location', 
                       'Merchant_Category', 'Authentication_Method']
numerical_features = [col for col in X.columns if col not in categorical_features]

#### Create preprocessing pipeline

In [35]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Saved the Preprocessed Dataset file For Further Use

In [37]:
df.to_csv('../assets/preprocessed_fraud_dataset.csv', index=False)